## Comparing benchmarking results of P100 vs V100

In [1]:
## How do the results compare from runs on different GPUs

## P100 Performance Analysis

In [2]:
#Going through metrics
import numbers
import glob
import os
import csv

if True:
    combined_data_ = {}
    target_kernels = []
    
metric_targets = []
all_sig_metrics = []
bench_targets = [ "gaussian" ,"gemm", "stream" ]
#for filen_ in glob.glob("/home/yzamora/power/nvidia_gpus/all_apps/mixed_results/*.csv"):
for filen_ in glob.glob("/home/yzamora/power/nvidia_gpus/all_apps/gaussian_results/*.csv"):
    filen = os.path.basename(filen_)
    #print (filen)
    filen_split = filen.split('.')[0].split('_')
    bench_name = filen_split[0]
    size = filen_split[1]
    #metric_name = filen_split[1]
    #print(filen_split)
    #size_str = filen_split[1].split('N')[1]
    
    #if not (metric_name in metric_targets): continue
    #if not (bench_name in bench_targets): continue
    
    key_root = bench_name
    #print(key_root)
    levels = ["Idle", "Low","High", "Max"]
    bw_units = ["GB", "MB", "KB" ,"0B"]
    # Now open the file and look for the data
    with open(filen_ ,'r') as file_handle:
        #print (file_handle)
        data_found = False
        ncols = 1
        fdata = csv.reader(file_handle)
        index_lookup = {}
        #print(filen_)
        for line_split in fdata:
            #print (line_split)
            lsplt = (len(line_split) > 0)
      
            if data_found:
                #print("data found")
                if lsplt and len(line_split) == ncols:
                    #percent - strip off end
                    # Get metric name here
                    #mname_index = index_lookup['Metric Name']
                    #metric_name = line_split[ mname_index ]
                    #if not (metric_name in metric_targets): continue
                    
                    # Read in desired value for the current metric
                    target_index = index_lookup['Avg']; value = 0
                    metric_name = line_split[index_lookup['Metric Name']]
                    #print (line_split[target_index].isdecimal())
                    if line_split[target_index].isdecimal():
                        if line_split[target_index]!= '0':
                            #print(line_split[target_index])
                            all_sig_metrics.append(metric_name)
                            value = int(line_split[ target_index ])
                        
                        # Labeled with percentage
                    elif "%" == line_split[target_index][-1]:
                        #print ("percentage loop")
                        all_sig_metrics.append(metric_name)
                        value = float(line_split[ target_index ][0:7]) / 100.0
                        
                    # Labeled with bandwidth units
                    elif line_split[ target_index ][-4:-2] in bw_units:
                        # Just take the first 
                        units = line_split[ target_index ][-4:-2]
                        all_sig_metrics.append(metric_name)
                        mfact = 1.0
                        if   units == "KB": mfact = 1024
                        elif units == "MB": mfact = 1024*1024
                        elif units == "GB": mfact = 1024*1024*1024
                        elif units == "0B":  mfact = 1
                        value = float(line_split[ target_index ][0:7]) * mfact
                    
                    # idle, low, max
                    elif line_split[ target_index ][-1] == ")":
                        #print ("low")
                        all_sig_metrics.append(metric_name)
                        value = int(line_split[ target_index].split('(')[1].split(")")[0])
                        
                    # otherwise, float
                    #elif not(float(line_split[ target_index ]).is_integer()):
                    else:
                        #print(line_split[ target_index ].split('(')[0])
                        #print("in float")
                        #print(line_split[ target_index ].split('(')[0])
                        value = float(line_split[ target_index ])
                    
                    
                    
                     # Parse name of kernel
                    kernel_name = line_split[ index_lookup['Kernel'] ].split('(')[0]
                    
                    # Define kernel-specific key
                    key = key_root + size + "_" + kernel_name 
                    
                    # Initialize dict for this key, if it is new
                    if not (key in combined_data_): 
                        combined_data_ [ key ] = {}
                    if not (kernel_name in target_kernels):
                        target_kernels.append(kernel_name)
                        #combined_data_ [ key ][ 'size' ] = int( size_str )
                        
                    # Store value for the metric being read right now
                    combined_data_ [key][ metric_name ] = value
                    combined_data_[key]["kernelname"] = kernel_name
                    
                else: data_found = False


            elif lsplt and line_split[0] == 'Device' and line_split[1] == 'Kernel':
                # Set flag that we are at the data:
                data_found = True
                # Set number of columns in table:
                ncols = len(line_split)
                # Generate an index lookup table:
                idx = 0
                for term in line_split:
                    index_lookup[term] = idx
                    idx += 1
                #print(index_lookup)


In [3]:
#Printing P100 Data 
p100_data = combined_data_
print (p100_data.keys())

dict_keys(['gaussianN32_Fan2', 'gaussianN32_Fan1', 'gaussianN64_Fan2', 'gaussianN64_Fan1', 'gaussianN96_Fan2', 'gaussianN96_Fan1', 'gaussianN128_Fan2', 'gaussianN128_Fan1', 'gaussianN160_Fan2', 'gaussianN160_Fan1', 'gaussianN192_Fan2', 'gaussianN192_Fan1', 'gaussianN224_Fan2', 'gaussianN224_Fan1', 'gaussianN256_Fan2', 'gaussianN256_Fan1', 'gaussianN288_Fan2', 'gaussianN288_Fan1', 'gaussianN320_Fan2', 'gaussianN320_Fan1', 'gaussianN352_Fan2', 'gaussianN352_Fan1', 'gaussianN384_Fan2', 'gaussianN384_Fan1', 'gaussianN416_Fan2', 'gaussianN416_Fan1', 'gaussianN448_Fan2', 'gaussianN448_Fan1', 'gaussianN480_Fan2', 'gaussianN480_Fan1', 'gaussianN512_Fan2', 'gaussianN512_Fan1', 'gaussianN544_Fan2', 'gaussianN544_Fan1', 'gaussianN576_Fan2', 'gaussianN576_Fan1', 'gaussianN608_Fan2', 'gaussianN608_Fan1', 'gaussianN640_Fan2', 'gaussianN640_Fan1', 'gaussianN672_Fan2', 'gaussianN672_Fan1', 'gaussianN704_Fan2', 'gaussianN704_Fan1', 'gaussianN736_Fan2', 'gaussianN736_Fan1', 'gaussianN768_Fan2', 'gaussia

## V100 Performance Analysis

In [4]:
#Going through V100 metrics
import numbers
import glob
import os
import csv

if True:
    combined_data_ = {}
    target_kernels = []
    
metric_targets = []
all_sig_metrics = []
bench_targets = [ "gaussian" ,"gemm", "stream" ]
for filen_ in glob.glob("/gpfs/jlse-fs0/users/yzamora/v100_testing/gaussian_results/*.csv"):
#for filen_ in glob.glob("/home/yzamora/power/nvidia_gpus/all_apps/gaussian_results/*.csv"):
    filen = os.path.basename(filen_)
    #print (filen)
    filen_split = filen.split('.')[0].split('_')
    bench_name = filen_split[0]
    size = filen_split[1]
    #metric_name = filen_split[1]
    #print(filen_split)
    #size_str = filen_split[1].split('N')[1]
    
    #if not (metric_name in metric_targets): continue
    #if not (bench_name in bench_targets): continue

    
    key_root = bench_name
    #print(key_root)
    levels = ["Idle", "Low","High", "Max"]
    bw_units = ["GB", "MB", "KB" ,"0B"]
    # Now open the file and look for the data
    with open(filen_ ,'r') as file_handle:
        #print (file_handle)
        data_found = False
        ncols = 1
        fdata = csv.reader(file_handle)
        index_lookup = {}
        #print(filen_)
        for line_split in fdata:
            #print (line_split)
            lsplt = (len(line_split) > 0)
    
            if data_found:
                #print("data found")
                if lsplt and len(line_split) == ncols:
                    #percent - strip off end
                    # Get metric name here
                    #mname_index = index_lookup['Metric Name']
                    #metric_name = line_split[ mname_index ]
                    #if not (metric_name in metric_targets): continue
                    
                    # Read in desired value for the current metric
                    target_index = index_lookup['Avg']; value = 0
                    metric_name = line_split[index_lookup['Metric Name']]
                    #print (line_split[target_index].isdecimal())
                    if line_split[target_index].isdecimal():
                        if line_split[target_index]!= '0':
                            #print(line_split[target_index])
                            all_sig_metrics.append(metric_name)
                            value = int(line_split[ target_index ])
                        
                        # Labeled with percentage
                    elif "%" == line_split[target_index][-1]:
                        #print ("percentage loop")
                        all_sig_metrics.append(metric_name)
                        value = float(line_split[ target_index ][0:7]) / 100.0
                        
                    # Labeled with bandwidth units
                    elif line_split[ target_index ][-4:-2] in bw_units:
                        # Just take the first 
                        units = line_split[ target_index ][-4:-2]
                        all_sig_metrics.append(metric_name)
                        mfact = 1.0
                        if   units == "KB": mfact = 1024
                        elif units == "MB": mfact = 1024*1024
                        elif units == "GB": mfact = 1024*1024*1024
                        elif units == "0B":  mfact = 1
                        value = float(line_split[ target_index ][0:7]) * mfact
                    
                    # idle, low, max
                    elif line_split[ target_index ][-1] == ")":
                        #print ("low")
                        all_sig_metrics.append(metric_name)
                        value = int(line_split[ target_index].split('(')[1].split(")")[0])
                        
                    # otherwise, float
                    #elif not(float(line_split[ target_index ]).is_integer()):
                    else:
                        #print(line_split[ target_index ].split('(')[0])
                        #print("in float")
                        #print(line_split[ target_index ].split('(')[0])
                        value = float(line_split[ target_index ])
                    
                    
                    
                     # Parse name of kernel
                    kernel_name = line_split[ index_lookup['Kernel'] ].split('(')[0]
                    
                    # Define kernel-specific key
                    key = key_root + size + "_" + kernel_name 
                    
                    # Initialize dict for this key, if it is new
                    if not (key in combined_data_): 
                        combined_data_ [ key ] = {}
                    if not (kernel_name in target_kernels):
                        target_kernels.append(kernel_name)
                        #combined_data_ [ key ][ 'size' ] = int( size_str )
                        
                    # Store value for the metric being read right now
                    combined_data_ [key][ metric_name ] = value
                    combined_data_[key]["kernelname"] = kernel_name
                    
                else: data_found = False


            elif lsplt and line_split[0] == 'Device' and line_split[1] == 'Kernel':
                # Set flag that we are at the data:
                data_found = True
                # Set number of columns in table:
                ncols = len(line_split)
                # Generate an index lookup table:
                idx = 0
                for term in line_split:
                    index_lookup[term] = idx
                    idx += 1
                #print(index_lookup)


In [5]:
#printing combined keys for V100 performance data
v100_data = combined_data_
print(v100_data.keys())

dict_keys(['gaussianN896_Fan2', 'gaussianN896_Fan1', 'gaussianN960_Fan2', 'gaussianN960_Fan1', 'gaussianN736_Fan2', 'gaussianN736_Fan1', 'gaussianN800_Fan2', 'gaussianN800_Fan1', 'gaussianN832_Fan2', 'gaussianN832_Fan1', 'gaussianN352_Fan2', 'gaussianN352_Fan1', 'gaussianN864_Fan2', 'gaussianN864_Fan1', 'gaussianN416_Fan2', 'gaussianN416_Fan1', 'gaussianN224_Fan2', 'gaussianN224_Fan1', 'gaussianN672_Fan2', 'gaussianN672_Fan1', 'gaussianN640_Fan2', 'gaussianN640_Fan1', 'gaussianN96_Fan2', 'gaussianN96_Fan1', 'gaussianN512_Fan2', 'gaussianN512_Fan1', 'gaussianN256_Fan2', 'gaussianN256_Fan1', 'gaussianN160_Fan2', 'gaussianN160_Fan1', 'gaussianN544_Fan2', 'gaussianN544_Fan1', 'gaussianN480_Fan2', 'gaussianN480_Fan1', 'gaussianN704_Fan2', 'gaussianN704_Fan1', 'gaussianN608_Fan2', 'gaussianN608_Fan1', 'gaussianN768_Fan2', 'gaussianN768_Fan1', 'gaussianN128_Fan2', 'gaussianN128_Fan1', 'gaussianN448_Fan2', 'gaussianN448_Fan1', 'gaussianN384_Fan2', 'gaussianN384_Fan1', 'gaussianN192_Fan2', 'gau

In [6]:
num_keys= 0
for k,v in v100_data.items():
    if k in p100_data.keys():
        num_keys +=1
print (num_keys) #number of similar kernels that will be compared

58


In [7]:
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        pass
 
    try:
        import unicodedata
        unicodedata.numeric(s)
        return True
    except (TypeError, ValueError):
        pass
 
    return False


In [8]:
def compare_metrics(dict1,dict2,x):
    compared ={}
    differences = {}
    diffkeys = []
    #dict1 = combined_data_['hotspot_calculate_temp']
    #dict2 = combined_data_['huffman_vlc_encode_kernel_sm64huff']
    if x:
        #diffkeys = [k for k in dict1 if dict1[k] != dict2[k]]
        for k in dict1:
            if dict1[k] != dict2[k]: diffkeys.append( k )
                
    else:
        #diffkeys = [k for k in dict1 if dict1[k] == dict2[k]]
        for k in dict1:
            if dict1[k] == dict2[k]: diffkeys.append( k )
                
    for k in diffkeys:
        #print ("k " + k)
        #print ("dict1 " + str(dict1[k]))
        #print ("dict2 " + str(dict2[k]))

        if is_number(dict1[k]) and is_number(dict2[k]):
            #print(dict1[k], dict2[k] )
            compared[k] =  dict1[k], dict2[k],abs(dict1[k]-dict2[k])
            if abs(dict1[k]-dict2[k]) != 0:
                   differences[k] = abs(dict1[k]-dict2[k])

    return (differences)

In [16]:
for k,v in v100_data.items():
    if k in p100_data.keys():
        #print(k)
        #print (p100_data['gaussianN896_Fan2'])
        #print('!!!!!P100 DATA!!!!!!')
        #print(v100_data['gaussianN896_Fan2'])
        #print(v100_data[k])
        print (compare_metrics(p100_data[k],v100_data[k], True))
#print (v100_data['gaussianN32_Fan2'])

{'inst_per_warp': 89.26674200000001, 'branch_efficiency': 0.003933999999999993, 'warp_execution_efficiency': 0.00037799999999998946, 'warp_nonpred_execution_efficiency': 0.009238999999999942, 'inst_replay_overhead': 0.000974, 'gld_transactions_per_request': 3.1157690000000002, 'gst_transactions_per_request': 4.296244, 'gld_transactions': 162527, 'gst_transactions': 75026, 'l2_read_transactions': 28622, 'l2_write_transactions': 74959, 'dram_read_transactions': 3208, 'dram_write_transactions': 2379, 'global_hit_rate': 0.013553000000000037, 'gld_requested_throughput': 2349561859.2768, 'gst_requested_throughput': 1563690218.2911987, 'gld_throughput': 44699442636.390396, 'gst_throughput': 20246261084.979202, 'local_memory_overhead': 0.44717300000000004, 'tex_cache_hit_rate': 0.15573099999999998, 'l2_tex_read_hit_rate': 0.12137300000000006, 'dram_read_throughput': 378816115.50720024, 'dram_write_throughput': 972058473.2671986, 'tex_cache_throughput': 210447706672.33282, 'l2_tex_read_throughp